In [1]:
#importing all the required libraries and modules
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from pandas.tseries.holiday import USFederalHolidayCalendar
import scipy.stats
import matplotlib.dates as mdates
from google.cloud import bigquery

In [2]:
#connecting notebook to gcloud bigquery to query data.
cred_file = r"C:\Users\V0V01S7\AppData\Roaming\gcloud\legacy_credentials\venkatasaiyashaswi.v@walmart.com\adc.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=cred_file
def query_bq(QUERY):
    client = bigquery.Client(project='wmt-edw-sandbox')
    query_job = client.query(QUERY) # API request
    rows = query_job.result() # Waits for query to finish
    df = rows.to_dataframe()
    return df

In [79]:
code = """select A.DRVR_USER_ID, 
B.ACTV_START_TS, B.ACTL_ACTV_END_TS, B.ACTV_DUR_MIN_QTY, B.ACTV_DT, 
from `wmt-edw-prod.WW_GEC_VM.DRVR` A inner join `wmt-edw-prod.WW_GEC_VM.DRVR_APPLN_ACTV` B on A.DRVR_USER_ID = B.DRVR_USER_ID
where A.MARKET_NM = 'Walmart Chicago' AND B.ACTV_NM = "ONLINE"
"""
df = query_bq(code)
df["ACTV_START_TS"] = pd.to_datetime(df["ACTV_START_TS"])
df["ACTL_ACTV_END_TS"] = pd.to_datetime(df["ACTL_ACTV_END_TS"])
df["ACTV_START_TS"] = df["ACTV_START_TS"] + pd.Timedelta(hours=2) #converting pacific time to eastern time (orlando market)
df["ACTL_ACTV_END_TS"] = df["ACTL_ACTV_END_TS"] + pd.Timedelta(hours=2)
len(df)

27691

In [80]:
df = df[df["DRVR_USER_ID"].str.contains("walmart") == False]
df.head()

,DRVR_USER_ID,ACTV_START_TS,ACTL_ACTV_END_TS,ACTV_DUR_MIN_QTY,ACTV_DT
0,gpeterson903@yahoo.com,2021-07-03 14:08:07.180,2021-07-03 14:10:05.728,1.970000000,2021-07-03
1,baybayturnipseed@icloud.com,2021-07-03 07:01:15.785,2021-07-03 11:00:00.000,238.750000000,2021-07-03
2,baybayturnipseed@icloud.com,2021-07-03 11:05:09.300,2021-07-03 14:49:03.917,223.900000000,2021-07-03
3,baybayturnipseed@icloud.com,2021-07-03 14:49:03.917,2021-07-03 19:00:00.000,250.950000000,2021-07-03
4,starksnicholas19@yahoo.com,2021-03-24 08:20:11.763,2021-03-24 11:30:00.000,189.800000000,2021-03-24


In [81]:
def final_ft(df):
    df['dayhour'] = df["ACTV_START_TS"].apply(lambda x: x.replace(minute=0, second=0, microsecond=0))
    time_line = pd.date_range(df["dayhour"].min(),df["dayhour"].max(),freq='H')
    time_line = time_line.to_series()
    ts = pd.DataFrame(time_line,columns = ["dayhour"])
    ts = ts.set_index("dayhour")
    ts = pd.merge(ts,df,how="left",on=["dayhour"])
    ts["ACTV_DUR_MIN_QTY"] = ts["ACTV_DUR_MIN_QTY"].fillna(0)
    ts = ts.drop(["DRVR_USER_ID","ACTV_DT"],axis=1)
    ts = ts.set_index(["dayhour"],drop=True)
    ts = ts.drop_duplicates()
    ts = ts.dropna()
    ts = ts.reset_index(drop=False)
    ts['ACTV_START_TS'] = ts["ACTV_START_TS"].apply(lambda x: x.replace(second=0, microsecond=0))
    ts['ACTL_ACTV_END_TS'] = ts["ACTL_ACTV_END_TS"].apply(lambda x: x.replace(second=0, microsecond=0))
    def create_final_data(df):
        df['minutes'] = (df['ACTL_ACTV_END_TS'] - df['ACTV_START_TS']).dt.total_seconds()/60
        df = df[df['minutes']>0].reset_index(drop=True)
        # create an IntervalIndex which we can set as the axis (needed for re-indexing).
        # subtract one minute from end_time so that the minute of the termination is excluded.
        iv_idx = pd.IntervalIndex.from_arrays(df['ACTV_START_TS'],
                                              df['ACTL_ACTV_END_TS']-pd.Timedelta(minutes=1),
                                              closed='both')
        # create a new index with the extended frequency:
        new_idx = pd.date_range(df['ACTV_START_TS'].min(), df['ACTL_ACTV_END_TS'].max(), freq='min')
        # set the new index to get the extended frequency;
        # all minutes will have the value of the whole interval
        result = df['minutes'].set_axis(iv_idx).reindex(new_idx)
        # we can now calculate the duration per hour by resampling and summing the
        # boolean representation of the duration (1/0):
        result= result.fillna(0).astype(int).astype(bool).resample('H').sum()
        result.index.name = 'start_hour'
        return result
    re = create_final_data(ts)
    re = pd.DataFrame(re)
    def create_features(df, label=None):
        """
        Creates time series features from datetime index.
        """
        df = df.copy()
        df['date'] = df.index
        df['hour'] = df['date'].dt.hour
        df['dayofweek'] = df['date'].dt.dayofweek
        df['quarter'] = df['date'].dt.quarter
        df['month'] = df['date'].dt.month
        df['year'] = df['date'].dt.year
        df['dayofyear'] = df['date'].dt.dayofyear
        df['dayofmonth'] = df['date'].dt.day
        df['weekofyear'] = df['date'].dt.isocalendar().week

        X = df[['hour','dayofweek','quarter','month','year',
               'dayofyear','dayofmonth','weekofyear']]
        if label:
            y = df[label]
            return X, y
        return X

    X, y = create_features(re, label='minutes')

    ft = pd.concat([X, y], axis=1)    
    return ft

In [82]:
dfs = {}
for ID in df['DRVR_USER_ID'].value_counts().index.tolist():
    dfs[ID] = df.loc[df['DRVR_USER_ID'] == ID]

In [83]:
len(dfs)

210

In [84]:
dfs_set = {}
for i in dfs.keys():
    dfs_set[i] = pd.DataFrame(final_ft(dfs[i]))
    dfs_set[i]["ID"] = i

C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_21004\3945249123.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dayhour'] = df["ACTV_START_TS"].apply(lambda x: x.replace(minute=0, second=0, microsecond=0))
C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_21004\3945249123.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dayhour'] = df["ACTV_START_TS"].apply(lambda x: x.replace(minute=0, second=0, microsecond=0))
C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_21004\3945249123.py:2: SettingWithCopyWa

C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_21004\3945249123.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dayhour'] = df["ACTV_START_TS"].apply(lambda x: x.replace(minute=0, second=0, microsecond=0))
C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_21004\3945249123.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dayhour'] = df["ACTV_START_TS"].apply(lambda x: x.replace(minute=0, second=0, microsecond=0))
C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_21004\3945249123.py:2: SettingWithCopyWa

C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_21004\3945249123.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dayhour'] = df["ACTV_START_TS"].apply(lambda x: x.replace(minute=0, second=0, microsecond=0))
C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_21004\3945249123.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dayhour'] = df["ACTV_START_TS"].apply(lambda x: x.replace(minute=0, second=0, microsecond=0))
C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_21004\3945249123.py:2: SettingWithCopyWa

C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_21004\3945249123.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dayhour'] = df["ACTV_START_TS"].apply(lambda x: x.replace(minute=0, second=0, microsecond=0))
C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_21004\3945249123.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dayhour'] = df["ACTV_START_TS"].apply(lambda x: x.replace(minute=0, second=0, microsecond=0))
C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_21004\3945249123.py:2: SettingWithCopyWa

C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_21004\3945249123.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dayhour'] = df["ACTV_START_TS"].apply(lambda x: x.replace(minute=0, second=0, microsecond=0))
C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_21004\3945249123.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dayhour'] = df["ACTV_START_TS"].apply(lambda x: x.replace(minute=0, second=0, microsecond=0))
C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_21004\3945249123.py:2: SettingWithCopyWa

...

C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_21004\3945249123.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dayhour'] = df["ACTV_START_TS"].apply(lambda x: x.replace(minute=0, second=0, microsecond=0))
C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_21004\3945249123.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dayhour'] = df["ACTV_START_TS"].apply(lambda x: x.replace(minute=0, second=0, microsecond=0))
C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_21004\3945249123.py:2: SettingWithCopyWa

C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_21004\3945249123.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dayhour'] = df["ACTV_START_TS"].apply(lambda x: x.replace(minute=0, second=0, microsecond=0))
C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_21004\3945249123.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dayhour'] = df["ACTV_START_TS"].apply(lambda x: x.replace(minute=0, second=0, microsecond=0))
C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_21004\3945249123.py:2: SettingWithCopyWa

C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_21004\3945249123.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dayhour'] = df["ACTV_START_TS"].apply(lambda x: x.replace(minute=0, second=0, microsecond=0))
C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_21004\3945249123.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dayhour'] = df["ACTV_START_TS"].apply(lambda x: x.replace(minute=0, second=0, microsecond=0))
C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_21004\3945249123.py:2: SettingWithCopyWa

C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_21004\3945249123.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dayhour'] = df["ACTV_START_TS"].apply(lambda x: x.replace(minute=0, second=0, microsecond=0))
C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_21004\3945249123.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dayhour'] = df["ACTV_START_TS"].apply(lambda x: x.replace(minute=0, second=0, microsecond=0))
C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_21004\3945249123.py:2: SettingWithCopyWa

ValueError: Neither `start` nor `end` can be NaT

In [85]:
dff = pd.concat([dfs_set[i] for i in dfs_set.keys()], ignore_index=False)

In [86]:
dff.head()

,hour,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,minutes,ID
start_hour,,,,,,,,,,
2021-09-15 11:00:00,11,2,3,9,2021,258,15,37,6,jkirk427@yahoo.com
2021-09-15 12:00:00,12,2,3,9,2021,258,15,37,60,jkirk427@yahoo.com
2021-09-15 13:00:00,13,2,3,9,2021,258,15,37,39,jkirk427@yahoo.com
2021-09-15 14:00:00,14,2,3,9,2021,258,15,37,0,jkirk427@yahoo.com
2021-09-15 15:00:00,15,2,3,9,2021,258,15,37,0,jkirk427@yahoo.com


In [87]:
def slot_calc(hour):
    
    if hour in range(6,12):
        return "morning"
    elif hour in range(12,17):
        return "afternoon"
    elif hour in range(17,22):
        return "evening"
    
def weekend_calc(dayofweek):
    if dayofweek in range(0,5):
        return "weekday"
    elif dayofweek in range(5,7):
        return "weekend"

cal = USFederalHolidayCalendar()
    
def holiday_calc(start_hour):
    if start_hour.date() in cal.holidays(start="2021-01-01", end=datetime.today()):
        return "holiday"
    else:
        return "non-holiday"

In [88]:
dff["slot"] = dff["hour"].apply(slot_calc)
dff["weekend"] = dff["dayofweek"].apply(weekend_calc)
#dff["start_hour"] = dff.index
#dff["holiday"] = dff["start_hour"].apply(holiday_calc)

In [89]:
dff.head()

,hour,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,minutes,ID,slot,weekend
start_hour,,,,,,,,,,,,
2021-09-15 11:00:00,11,2,3,9,2021,258,15,37,6,jkirk427@yahoo.com,morning,weekday
2021-09-15 12:00:00,12,2,3,9,2021,258,15,37,60,jkirk427@yahoo.com,afternoon,weekday
2021-09-15 13:00:00,13,2,3,9,2021,258,15,37,39,jkirk427@yahoo.com,afternoon,weekday
2021-09-15 14:00:00,14,2,3,9,2021,258,15,37,0,jkirk427@yahoo.com,afternoon,weekday
2021-09-15 15:00:00,15,2,3,9,2021,258,15,37,0,jkirk427@yahoo.com,afternoon,weekday


In [90]:
wdff = pd.DataFrame(dff.groupby(by = ["ID","weekend"])["minutes"].mean())

In [91]:
sdff = pd.DataFrame(dff.groupby(by = ["ID","slot"])["minutes"].mean())

In [117]:
wdff

minutes
ID                         weekend          
1985chicitybears@gmail.com weekday  0.885770
                           weekend  0.183989
19detroit93@gmail.com      weekend  1.000000
abrlsc460@gmail.com        weekday  9.500000
acanton29@gmail.com        weekday  0.157691
...                                      ...
xxjuliomedinaxx@gmail.com  weekend  0.064516
yasminehadi1801@gmail.com  weekday  0.268222
                           weekend  0.210648
yungisdaname@gmail.com     weekday  4.500877
                           weekend  2.625000

[368 rows x 1 columns]

In [125]:
print(wdff[wdff.minutes > 20].shape)
wdff[wdff.minutes > 20]

(18, 1)


minutes
ID                            weekend           
athenaanderson@ymail.com      weekday  43.000000
carlegbuhuzo@gmail.com        weekday  53.428571
dlozan8@uic.edu               weekday  38.666667
eril_vega@yahoo.com           weekday  46.600000
escobarmario26@yahoo.com      weekday  21.500000
fo.6969.fo@gmail.com          weekday  38.600000
jacquelinegamble7@gmail.com   weekday  31.004230
                              weekend  27.613426
javonnakidd@yahoo.com         weekend  47.200000
jbrock32@gmail.com            weekday  45.500000
keshawndaeggleston5@gmail.com weekend  25.333333
kusek.megan@yahoo.com         weekend  45.200000
marshanequiabraxton@yahoo.com weekday  42.000000
mplo10ent@gmail.com           weekday  43.250000
myersrobert412@gmail.com      weekday  48.750000
pokeywalker1153@icloud.com    weekday  37.666667
richardw903@gmail.com         weekend  33.000000
tatiananathan@yahoo.com       weekend  40.250000

In [140]:
print(ww[ww.minutes > 20].shape)
ww[ww.minutes > 20]

(17, 2)


,,minutes,location
ID,weekend,,
athenaanderson@ymail.com,weekday,43.000000,Walmart San Jose North
carlegbuhuzo@gmail.com,weekday,53.428571,Walmart San Jose North
dlozan8@uic.edu,weekday,38.666667,Walmart San Jose North
eril_vega@yahoo.com,weekday,46.600000,Walmart San Jose North
escobarmario26@yahoo.com,weekday,21.500000,Walmart San Jose North
fo.6969.fo@gmail.com,weekday,38.600000,Walmart San Jose North
jacquelinegamble7@gmail.com,weekday,31.004230,Walmart San Jose North
javonnakidd@yahoo.com,weekend,47.200000,Walmart San Jose North
jbrock32@gmail.com,weekday,45.500000,Walmart San Jose North


In [146]:
ww = ww.reset_index(level=['weekend'])
ww.head()

KeyError: 'Requested level (weekend) does not match index name (ID)'

In [148]:
ww

,weekend,minutes,location
ID,,,
1985chicitybears@gmail.com,weekday,0.885770,Walmart San Jose North
19detroit93@gmail.com,weekend,1.000000,Walmart San Jose North
abrlsc460@gmail.com,weekday,9.500000,Walmart San Jose North
acanton29@gmail.com,weekday,0.157691,Walmart San Jose North
adrian.treyo@gmail.com,weekday,0.033195,Walmart San Jose North
...,...,...,...
webbecstasy@yahoo.com,weekend,0.152167,Walmart San Jose North
wrickslisa@yahoo.com,weekday,0.179820,Walmart San Jose North
xxjuliomedinaxx@gmail.com,weekday,0.696629,Walmart San Jose North


In [142]:
ww.columns

Index(['minutes', 'location'], dtype='object')

In [93]:
ww = wdff.loc[wdff.groupby('ID')['minutes'].idxmax()]

In [94]:
sod = sdff.loc[sdff.groupby('ID')['minutes'].idxmax()]

In [149]:
sod = sod.reset_index(level = ["slot"])

In [150]:
sod

,slot,minutes,location
ID,,,
1985chicitybears@gmail.com,afternoon,1.488417,Walmart San Jose North
19detroit93@gmail.com,morning,1.000000,Walmart San Jose North
abrlsc460@gmail.com,morning,9.500000,Walmart San Jose North
acanton29@gmail.com,morning,0.494283,Walmart San Jose North
adrian.treyo@gmail.com,evening,0.057143,Walmart San Jose North
...,...,...,...
webbecstasy@yahoo.com,afternoon,0.302092,Walmart San Jose North
wrickslisa@yahoo.com,afternoon,0.427165,Walmart San Jose North
xxjuliomedinaxx@gmail.com,afternoon,2.137931,Walmart San Jose North


In [95]:
ww.to_csv("Desktop/ww_chic.csv")
sod.to_csv("Desktop/sod_chic.csv")

In [101]:
hdff = pd.DataFrame(dff.groupby(by = ["ID","hour"])["minutes"].mean())
hdff.loc[hdff.groupby('ID')['minutes'].idxmax()].to_csv("Desktop/hourly_chicago.csv")

In [116]:
ww["location"] = "Walmart San Jose North"
sod["location"] = "Walmart San Jose North"

In [152]:
code2 = """select *
from `wmt-edw-prod.WW_GEC_VM.DRVR` 
where MARKET_NM = 'Walmart Chicago'
"""
df2 = query_bq(code2)

In [158]:
df2

,DRVR_USER_ID,DRVR_CITY_OF_INT_NM,DRVR_CITY_NM,DRVR_ST_NM,DRVR_ZIPCD,PYMT_REGTN_ID,DRVR_ACTV_IND,DRVR_ONBDG_ST_CD,REF_DRVR_USER_ID,DRVR_ONBDG_TS_UTC,...,LAST_ACTIVATED_TS,LAST_DEATVD_TS_UTC,LAST_DEATVD_TS,DEACTIVATION_RSN_NM,CREATE_TS_UTC,CREATE_TS,UPD_TS_UTC,UPD_TS,CREATE_USER,UPD_USER
0,savannahcox2010@yahoo.com,chicago,None,None,None,8b9039957cf9144b017d0afc34ff5e70,0,ONBOARDING_SUCCESS,None,2021-11-10 17:54:01+00:00,...,2021-11-10 09:54:00.730,2021-11-28 23:54:53.393000+00:00,2021-11-28 15:54:53.393,Voluntary,2021-11-11 10:05:20.696252+00:00,2021-11-11 02:05:20.696252,2021-11-29 10:04:48.424766+00:00,2021-11-29 02:04:48.424766,bfdms-sa-ecomm-lmd-prod@wmt-ae72e7270dedc6536a...,bfdms-sa-ecomm-lmd-prod@wmt-ae72e7270dedc6536a...
1,mecoles86@gmail.com,chicago,None,None,None,8aae6cfa7ad202f4017ae4c03bf16a2b,0,ONBOARDING_SUCCESS,None,2021-07-26 21:37:22+00:00,...,2021-07-26 14:37:21.607,2021-12-07 00:59:55.847000+00:00,2021-12-06 16:59:55.847,Behavior,2021-07-27 09:05:04.796871+00:00,2021-07-27 02:05:04.796871,2021-12-07 10:07:53.389070+00:00,2021-12-07 02:07:53.389070,bfdms-sa-ecomm-lmd-prod-sec@wmt-ae72e7270dedc6...,bfdms-sa-ecomm-lmd-prod@wmt-ae72e7270dedc6536a...
2,sparkwalmart1@gmail.com,chicago,None,None,None,8aa531656d8b49e6016d8e76f26c0702,0,ONBOARDING_SUCCESS,None,2019-10-02 21:53:41+00:00,...,2021-09-29 09:21:35.567,2021-11-15 12:00:02.230000+00:00,2021-11-15 04:00:02.230,Automatic,2021-09-30 11:16:13.473742+00:00,2021-09-30 04:16:13.473742,2021-12-11 10:06:16.817733+00:00,2021-12-11 02:06:16.817733,bfdms-sa-ecomm-lmd-prod@wmt-ae72e7270dedc6536a...,bfdms-sa-ecomm-lmd-prod@wmt-ae72e7270dedc6536a...
3,diego.sixtos.hernandez@gmail.com,chicago,None,None,None,8aae5c907af9ae98017b2bbc6e22769d,0,ONBOARDING_SUCCESS,None,2021-08-09 16:26:15+00:00,...,2021-08-09 09:26:14.647,2021-12-15 19:51:06.320000+00:00,2021-12-15 11:51:06.320,Vacation,2021-08-10 09:06:37.741106+00:00,2021-08-10 02:06:37.741106,2021-12-16 10:07:01.578826+00:00,2021-12-16 02:07:01.578826,bfdms-sa-ecomm-lmd-prod-sec@wmt-ae72e7270dedc6...,bfdms-sa-ecomm-lmd-prod@wmt-ae72e7270dedc6536a...
4,rosslawra24@icloud.com,chicago,None,None,None,8aae6cfa7c2f0e94017c4d08a01d3fc9,0,ONBOARDING_SUCCESS,None,2021-10-04 20:39:44+00:00,...,2021-10-04 13:39:43.633,2021-12-27 17:41:39.297000+00:00,2021-12-27 09:41:39.297,Behavior,2021-10-05 09:04:50.845076+00:00,2021-10-05 02:04:50.845076,2021-12-28 10:05:31.299198+00:00,2021-12-28 02:05:31.299198,bfdms-sa-ecomm-lmd-prod@wmt-ae72e7270dedc6536a...,bfdms-sa-ecomm-lmd-prod@wmt-ae72e7270dedc6536a...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
388,loubertharogers@yahoo.com,chicago,None,None,None,8aaef0da7815d57001781d56d33c3819,0,ONBOARDING_SUCCESS,None,2021-03-10 18:12:13+00:00,...,2021-03-10 10:12:12.830,2021-10-07 15:11:10.180000+00:00,2021-10-07 08:11:10.180,Vacation,2021-03-11 10:30:20.280751+00:00,2021-03-11 02:30:20.280751,2021-10-08 09:05:30.604067+00:00,2021-10-08 02:05:30.604067,bfdms-sa-ecomm-lmd-prod-sec@wmt-ae72e7270dedc6...,bfdms-sa-ecomm-lmd-prod@wmt-ae72e7270dedc6536a...
389,bhenry6834@gmail.com,chicago,None,None,None,8aae01f07c2f0e77017c4bea92f31523,0,ONBOARDING_SUCCESS,None,2021-10-04 15:27:17+00:00,...,2021-10-04 08:27:16.993,2021-10-28 11:32:41.607000+00:00,2021-10-28 04:32:41.607,Behavior,2021-10-05 09:04:50.845076+00:00,2021-10-05 02:04:50.845076,2021-10-29 09:04:53.307729+00:00,2021-10-29 02:04:53.307729,bfdms-sa-ecomm-lmd-prod@wmt-ae72e7270dedc6536a...,bfdms-sa-ecomm-lmd-prod@wmt-ae72e7270dedc6536a...
390,fahmed0@walmart.com,chicago,None,None,None,8aaee6a36cf1ad6b016cf8ebdf880c39,0,ONBOARDING_SUCCESS,None,2019-09-03 20:58:23+00:00,...,2021-08-23 12:17:49.823,2021-11-01 17:07:57.137000+00:00,2021-11-01 10:07:57.137,Behavior,2021-08-24 09:18:13.590416+00:00,2021-08-24 02:18:13.590416,2021-11-02 09:05:44.212907+00:00,2021-11-02 02:05:44.212907,bfdms-sa-ecomm-lmd-prod-sec@wmt-ae72e7270dedc6...,bfdms-sa-ecomm-lmd-prod@wmt-ae72e7270dedc6536a...
391,wchristopher29@gmail.com,ch

In [164]:
mer = pd.merge(ww,df2, left_on = ww.index, right_on = df2["DRVR_USER_ID"])

In [173]:
mer

,key_0,weekend,minutes,location,DRVR_USER_ID,DRVR_CITY_OF_INT_NM,DRVR_CITY_NM,DRVR_ST_NM,DRVR_ZIPCD,PYMT_REGTN_ID,...,LAST_ACTIVATED_TS,LAST_DEATVD_TS_UTC,LAST_DEATVD_TS,DEACTIVATION_RSN_NM,CREATE_TS_UTC,CREATE_TS,UPD_TS_UTC,UPD_TS,CREATE_USER,UPD_USER
0,1985chicitybears@gmail.com,weekday,0.885770,Walmart San Jose North,1985chicitybears@gmail.com,chicago,None,None,None,8aaeeedd77cd9394017803338b921208,...,2021-03-05 08:23:33.100,2021-07-05 05:05:05.370000+00:00,2021-07-04 22:05:05.370,Automatic,2021-03-06 06:11:26.061132+00:00,2021-03-05 22:11:26.061132,2021-07-05 09:05:03.783818+00:00,2021-07-05 02:05:03.783818,bfdms-sa-ecomm-lmd-prod-sec@wmt-ae72e7270dedc6...,bfdms-sa-ecomm-lmd-prod-sec@wmt-ae72e7270dedc6...
1,19detroit93@gmail.com,weekend,1.000000,Walmart San Jose North,19detroit93@gmail.com,chicago,None,None,None,8aaee4d07886fa3e0178895effd50543,...,2021-03-31 10:40:07.850,2021-07-26 05:04:42.500000+00:00,2021-07-25 22:04:42.500,Automatic,2021-04-01 09:05:54.972380+00:00,2021-04-01 02:05:54.972380,2021-07-26 09:03:56.455568+00:00,2021-07-26 02:03:56.455568,bfdms-sa-ecomm-lmd-prod-sec@wmt-ae72e7270dedc6...,bfdms-sa-ecomm-lmd-prod-sec@wmt-ae72e7270dedc6...
2,abrlsc460@gmail.com,weekday,9.500000,Walmart San Jose North,abrlsc460@gmail.com,chicago,None,None,None,8aae70e1781fd7d00178419d7b915718,...,2021-03-17 12:15:43.207,2021-07-12 05:05:20.517000+00:00,2021-07-11 22:05:20.517,Automatic,2021-03-18 09:07:54.378588+00:00,2021-03-18 02:07:54.378588,2021-07-12 09:05:08.172148+00:00,2021-07-12 02:05:08.172148,bfdms-sa-ecomm-lmd-prod-sec@wmt-ae72e7270dedc6...,bfdms-sa-ecomm-lmd-prod-sec@wmt-ae72e7270dedc6...
3,acanton29@gmail.com,weekday,0.157691,Walmart San Jose North,acanton29@gmail.com,chicago,None,None,None,8aaee4d07815d59901781967f99120db,...,2021-03-09 15:52:27.940,2021-07-05 05:04:46.267000+00:00,2021-07-04 22:04:46.267,Automatic,2021-03-10 10:06:10.827369+00:00,2021-03-10 02:06:10.827369,2021-07-05 09:05:03.783818+00:00,2021-07-05 02:05:03.783818,bfdms-sa-ecomm-lmd-prod-sec@wmt-ae72e7270dedc6...,bfdms-sa-ecomm-lmd-prod-sec@wmt-ae72e7270dedc6...
4,adrian.treyo@gmail.com,weekday,0.033195,Walmart San Jose North,adrian.treyo@gmail.com,chicago,None,None,None,8aae8cf0781fdd62017841f986800f79,...,2021-03-17 13:56:15.317,2021-07-12 05:05:19.227000+00:00,2021-07-11 22:05:19.227,Automatic,2021-03-18 09:07:54.378588+00:00,2021-03-18 02:07:54.378588,2021-07-12 09:05:08.172148+00:00,2021-07-12 02:05:08.172148,bfdms-sa-ecomm-lmd-prod-sec@wmt-ae72e7270dedc6...,bfdms-sa-ecomm-lmd-prod-sec@wmt-ae72e7270dedc6...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,webbecstasy@yahoo.com,weekend,0.152167,Walmart San Jose North,webbecstasy@yahoo.com,chicago,None,None,None,8aaef55d77f773650177fa4bc20a1e87,...,2021-03-03 14:53:25.007,2021-06-28 05:04:45.233000+00:00,2021-06-27 22:04:45.233,Automatic,2021-03-04 10:07:27.390934+00:00,2021-03-04 02:07:27.390934,2021-06-28 09:04:14.015545+00:00,2021-06-28 02:04:14.015545,bfdms-sa-ecomm-lmd-prod-sec@wmt-ae72e7270dedc6...,bfdms-sa-ecomm-lmd-prod-sec@wmt-ae72e7270dedc6...
202,wrickslisa@yahoo.com,weekday,0.179820,Walmart San Jose North,wrickslisa@yahoo.com,chicago,None,None,None,8aa58c3f78fcfbe701791ea9ee1f7bfc,...,2021-04-29 10:25:23.693,2021-08-23 05:04:08.607000+00:00,2021-08-22 22:04:08.607,Automatic,2021-04-30 09:14:43.181680+00:00,2021-04-30 02:14:43.181680,2021-08-23 09:05:03.974115+00:00,2021-08-23 02:05:03.974115,bfdms-sa-ecomm-lmd-prod-sec@wmt-ae72e7270dedc6...,bfdms-sa-ecomm-lmd-prod-sec@wmt-ae72e7270dedc6...
203,xxjuliomedinaxx@gmail.com,weekday,0.696629,Walmart San Jose North,xxjuliomedinaxx@gmail.com,chicago,None,None,None,8aae5c3377cd93c90177fabb5335783e,...,2021-03-03 16:55:16.713,2021-06-28 05:04:43.190000+00:00,2021-06-27 22:04:43.190,Automatic,2021-03-04 10:07:27.390934+00:00,2021-03-04 02:07:27.390934,2021-06-28 09:04:14.015545+00:00,2021-06-28 02:04:14.015545,bfdms-sa-ecomm-lmd-prod-sec@wmt-ae72e7270dedc6...,bfdms-sa-ecomm-lmd-prod-sec@wmt-ae72e

In [177]:
code3 = """select SALES_ORDER_NUM, DRVR_USER_ID, DLVR_OFFR_STS_NM
from `wmt-edw-prod.WW_GEC_VM.CARRIER_DLVR_OFFR`
where DLVR_OFFR_STS_NM = "ACCEPTED"  """
df3 = query_bq(code3)
df3.head()

KeyboardInterrupt: 